## Import

In [2]:
import numpy as np
from make_test_data import make_test_image
from photutils.isophote import Ellipse, EllipseGeometry

#old function
from model_old import build_ellipse_model as orig_build_ellipse_model

#new function
from model import build_ellipse_model as fast_build_ellipse_model

## Make simulated data, fit isophote

In [3]:
# make test image, fit isophote
data = make_test_image(nx=2000, ny=2000, i0=100., sma=500., eps=0.3,
                           pa=np.pi/4., noise=0.05, seed=0)
g = EllipseGeometry(1000., 1000., 100., 0.3, np.pi/4.)
ellipse = Ellipse(data, geometry=g, threshold=1.e5)
isophote_list = ellipse.fit_image()

In [18]:
# statistics of data
print("Average of data: " + str(np.average(data)))
print("Median of data: " + str(np.median(data)))
print("Max of data: " + str(np.max(data)))
print("Min of data: " + str(np.min(data)))

Average of data: 175.11916123012423
Median of data: 126.09453678854926
Max of data: 38861.20788934868
Min of data: 103.96731669580797


## Build model

In [5]:
%%time
# build using original model function
model_orig = orig_build_ellipse_model(data.shape, isophote_list, fill=np.mean(data[0:50, 0:50]))

CPU times: user 11min 52s, sys: 106 ms, total: 11min 52s
Wall time: 11min 52s


In [12]:
%%time
# build using new function, on 1 thread
model_fast_1thread = fast_build_ellipse_model(data.shape, isophote_list, nthreads=1, fill=np.mean(data[0:50, 0:50]))

CPU times: user 6.64 s, sys: 20 ms, total: 6.66 s
Wall time: 6.66 s


In [13]:
# check difference between model_orig & model_fast_1thread
print("Frobenius norm difference: " + str(np.linalg.norm(model_orig - model_fast_1thread)))
print("|Max difference|: " + str(np.max(np.abs(model_orig - model_fast_1thread))))

Frobenius norm difference: 0.0
|Max difference|: 0.0


=> No difference between old and new model, using 1 thread

In [14]:
%%time
# build using new function, on 8 thread
model_fast_8thread = fast_build_ellipse_model(data.shape, isophote_list, nthreads=8, fill=np.mean(data[0:50, 0:50]))

CPU times: user 9.49 s, sys: 620 ms, total: 10.1 s
Wall time: 2.27 s


In [15]:
# check difference between model_orig & model_fast_1thread
print("Frobenius norm difference: " + str(np.linalg.norm(model_orig - model_fast_8thread)))
print("|Max difference|: " + str(np.max(np.abs(model_orig - model_fast_8thread))))

Frobenius norm difference: 1.3448026978449904e-11
|Max difference|: 7.958078640513122e-13


=> very small difference, which is good considering the large magnitude of the data array